# Normalization of edge features for intergraph comparation of the effect of edge relaxation
When considering edge features in a graph such as $\delta ec = ec_{old} - ec_{new}$, one should be careful when using this parameter to compare different graphs, as for this particular example, if one graph has 1000 edge crossings and relaxing a certain edge removes one of them, we can consider it played little to no effect, however if another graph has 10 crossings and we remove one, that is a bettering of 10% compared ti the 0.1% seen before. And the fact is that the parameter $\Delta e_c$ conveys the same information in both cases.
We will therefore see a number of possible normalization processes for some of the edge attributes that we are dealing with.

## Edge crossings
Edge crossings are an inherent characteristic to a graph's drawing. They range from 0 to $\frac{(|E|-1)\cdot|E|}{2}$, and therefore the different in edge crossings ranges from - $\frac{(|E|-1)\cdot|E|}{2}$ to $\frac{(|E|-1)\cdot|E|}{2}$. It follows that a possible normalization procedure would be to divide $\delta e_c$ by $\frac{(|E|-1)\cdot|E|}{2}$, so as to get a value between -1 and 1.
Another possibility we can look at, which takes into account the graph's complexity is the reduction percentage, that we can define as $\frac{(ec_{old}-ec_{new})}{ec_{old}}$ which is close to 1 when we reduce the number of crossings significantly, 0 when we don't change it and negative when we worsen it. It's interesting to consider this variant as it allows for a qualitative improvement on the graph, not only numerical: we take into account how much has the graph gotten better, not only by how much it has changed.

## Expansion factor
This is a measure of disturbance of the drawing caused by he relaxation of an edge. It measures the distance between nodes before relaxing and nodes after relaxing and executing a couple more iterations of your favorite graph drawing algorithm (we will use Kamada Kawai). However, setting it to $\sum_{i=0}^{|V|}||x^{i}-f(x^{i})||$ allows bigger graphs with more nodes to have this parameter unusually high.
Therefore, we can consider normalizing in terms of the number of nodes: $\sum_{i=0}^{|V|}\frac{||x^{i}-f(x^{i})||}{|V|}$.
This way we have the disturbance per node and we don't discriminate in terms of the number of nodes of a graph.

## Gradient difference
We also measure the difference between the gradients of the Kamada Kawai before relaxing the edge and after. Note that because the non linear optimization looks for local optima, the gradient of the KK will possibly be zero after the execution and we measure the norm of the gradient after relaxing an edge. (Recomputing d(i,j)).
Therefore, our parameter could be $||\nabla(KK_{after}) - \nabla(KK_{before})||$. 

## Max degree of connecting nodes
This feature obviously increases as the size of our graph increases, and therefore it is a good idea to normalize it by dividing it by the total amount of nodes: $max_{v_i \in N_u}\frac{(deg(v_i)}{|V|}$

## Sum of degrees of connecting nodes
Similarly as for the max degree of connecting nodes, this feature increases with the size of our graph, and we therefore normalize it by dividing by the total amount of nodes: $\sum_{v_i \in N_u}\frac{deg(v_i)}{|V|}$